# Splitting the images into training & testing

Note: The images inside images don’t need to be split, only the .xml files.

---------data
            |----annotations
            |----images


In [2]:
%cd /home/subzero/Documents/0.TensorflowTrain/RaccoonDetector/data

/home/subzero/Documents/0.TensorflowTrain/RaccoonDetector/data


In [ ]:
#creating two dir for training and testing
!mkdir test_labels train_labels

In [3]:
# lists the files inside 'annotations' in a random order (not really random, by their hash value instead)
# Moves the first 400/2000 labels (20% of the labels) to the testing dir: `test_labels`
!ls annotations/* | sort -R | head -40 | xargs -I{} mv {} test_labels/

# Moves the rest of labels '1600' labels to the training dir: `train_labels`
!ls annotations/* | xargs -I{} mv {} train_labels/

sort: fflush failed: 'standard output': Broken pipe
sort: write error


# Installing Required Packages.

In [2]:
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -qq Cython contextlib2 pillow lxml matplotlib pycocotools
!pip install --upgrade tf-slim

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
  Using cached tf_slim-1.1.0-py2.py3-none-any.whl (352 kB)


# Importing Libraries

In [4]:
from __future__ import division, print_function, absolute_import

import pandas as pd
import numpy as np
import csv

import re
import os
import io
import glob
import shutil
import urllib.request
import tarfile
import xml.etree.ElementTree as ET

import tensorflow.compat.v1 as tf
import cv2 

from PIL import Image
from collections import namedtuple, OrderedDict

In [5]:
print(tf.__version__)

1.15.0


# Create CSV

In [6]:
#adjusted from: https://github.com/datitran/raccoon_dataset
def xml_to_csv(path):
  classes_names = []
  xml_list = []

  for xml_file in glob.glob(path + '/*.xml'):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for member in root.findall('object'):
      classes_names.append(member[0].text)
      value = (root.find('filename').text,
               int(root.find('size')[0].text),
               int(root.find('size')[1].text),
               member[0].text,
               int(member[4][0].text),
               int(member[4][1].text),
               int(member[4][2].text),
               int(member[4][3].text))
      xml_list.append(value)
  column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
  xml_df = pd.DataFrame(xml_list, columns=column_name) 
  classes_names = list(set(classes_names))
  classes_names.sort()
  return xml_df, classes_names

for label_path in ['train_labels', 'test_labels']:
  image_path = os.path.join(os.getcwd(), label_path)
  xml_df, classes = xml_to_csv(label_path)
  xml_df.to_csv(f'{label_path}.csv', index=None)
  print(f'Successfully converted {label_path} xml to csv.')

label_map_path = os.path.join("label_map.pbtxt")
pbtxt_content = ""

for i, class_name in enumerate(classes):
    pbtxt_content = (
        pbtxt_content
        + "item {{\n    id: {0}\n    name: '{1}'\n}}\n\n".format(i + 1, class_name)
    )
pbtxt_content = pbtxt_content.strip()
with open(label_map_path, "w") as f:
    f.write(pbtxt_content)

Successfully converted train_labels xml to csv.
Successfully converted test_labels xml to csv.


# Downloading Tensorflow model

In [7]:
%cd ..

/home/subzero/Documents/0.TensorflowTrain/RaccoonDetector


In [7]:
# downloads the models
!git clone --q https://github.com/tensorflow/models.git

In [8]:
%cd models/research

/home/subzero/Documents/0.TensorflowTrain/RaccoonDetector/models/research


In [9]:
# compils the proto buffers
!protoc object_detection/protos/*.proto --python_out=.

object_detection/protos/input_reader.proto:5:1: warning: Import object_detection/protos/image_resizer.proto is unused.


In [14]:
!python3 setup.py build
!python3 setup.py install

running build
running build_py
creating build
creating build/lib
creating build/lib/object_detection
copying object_detection/model_main_tf2.py -> build/lib/object_detection
copying object_detection/model_hparams.py -> build/lib/object_detection
copying object_detection/model_lib.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph_lib_tf1_test.py -> build/lib/object_detection
copying object_detection/inputs_test.py -> build/lib/object_detection
copying object_detection/model_lib_tf1_test.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph.py -> build/lib/object_detection
copying object_detection/exporter_main_v2.py -> build/lib/object_detection
copying object_detection/eval_util_test.py -> build/lib/object_detection
copying object_detection/__init__.py -> build/lib/object_detection
copying object_detection/exporter_lib_v2.py -> build/lib/object_detection
copying object_detection/eval_util.py -> build/lib/object_detection
copyi

copying object_detection/core/prefetcher_tf1_test.py -> build/lib/object_detection/core
copying object_detection/core/region_similarity_calculator_test.py -> build/lib/object_detection/core
creating build/lib/object_detection/inference
copying object_detection/inference/__init__.py -> build/lib/object_detection/inference
copying object_detection/inference/detection_inference.py -> build/lib/object_detection/inference
copying object_detection/inference/detection_inference_tf1_test.py -> build/lib/object_detection/inference
copying object_detection/inference/infer_detections.py -> build/lib/object_detection/inference
creating build/lib/object_detection/matchers
copying object_detection/matchers/argmax_matcher.py -> build/lib/object_detection/matchers
copying object_detection/matchers/bipartite_matcher_tf1_test.py -> build/lib/object_detection/matchers
copying object_detection/matchers/bipartite_matcher.py -> build/lib/object_detection/matchers
copying object_detection/matchers/__init__.p

creating object_detection.egg-info
writing object_detection.egg-info/PKG-INFO
writing dependency_links to object_detection.egg-info/dependency_links.txt
writing requirements to object_detection.egg-info/requires.txt
writing top-level names to object_detection.egg-info/top_level.txt
writing manifest file 'object_detection.egg-info/SOURCES.txt'
reading manifest file 'object_detection.egg-info/SOURCES.txt'
writing manifest file 'object_detection.egg-info/SOURCES.txt'
running install
running bdist_egg
running egg_info
writing object_detection.egg-info/PKG-INFO
writing dependency_links to object_detection.egg-info/dependency_links.txt
writing requirements to object_detection.egg-info/requires.txt
writing top-level names to object_detection.egg-info/top_level.txt
reading manifest file 'object_detection.egg-info/SOURCES.txt'
writing manifest file 'object_detection.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/

copying build/lib/object_detection/protos/post_processing_pb2.py -> build/bdist.linux-x86_64/egg/object_detection/protos
copying build/lib/object_detection/protos/keypoint_box_coder_pb2.py -> build/bdist.linux-x86_64/egg/object_detection/protos
copying build/lib/object_detection/protos/box_coder_pb2.py -> build/bdist.linux-x86_64/egg/object_detection/protos
copying build/lib/object_detection/protos/box_predictor_pb2.py -> build/bdist.linux-x86_64/egg/object_detection/protos
copying build/lib/object_detection/protos/losses_pb2.py -> build/bdist.linux-x86_64/egg/object_detection/protos
copying build/lib/object_detection/protos/image_resizer_pb2.py -> build/bdist.linux-x86_64/egg/object_detection/protos
copying build/lib/object_detection/protos/anchor_generator_pb2.py -> build/bdist.linux-x86_64/egg/object_detection/protos
copying build/lib/object_detection/protos/graph_rewriter_pb2.py -> build/bdist.linux-x86_64/egg/object_detection/protos
copying build/lib/object_detection/protos/ssd_an

byte-compiling build/bdist.linux-x86_64/egg/object_detection/predictors/convolutional_keras_box_predictor.py to convolutional_keras_box_predictor.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/object_detection/predictors/mask_rcnn_keras_box_predictor_tf2_test.py to mask_rcnn_keras_box_predictor_tf2_test.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/object_detection/predictors/mask_rcnn_box_predictor.py to mask_rcnn_box_predictor.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/object_detection/predictors/mask_rcnn_keras_box_predictor.py to mask_rcnn_keras_box_predictor.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/object_detection/predictors/rfcn_keras_box_predictor_tf2_test.py to rfcn_keras_box_predictor_tf2_test.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/object_detection/predictors/convolutional_box_predictor_tf1_test.py to convolutional_box_predictor_tf1_test.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/ob

byte-compiling build/bdist.linux-x86_64/egg/object_detection/builders/target_assigner_builder_test.py to target_assigner_builder_test.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/object_detection/builders/calibration_builder_test.py to calibration_builder_test.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/object_detection/builders/hyperparams_builder.py to hyperparams_builder.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/object_detection/builders/hyperparams_builder_test.py to hyperparams_builder_test.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/object_detection/builders/anchor_generator_builder_test.py to anchor_generator_builder_test.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/object_detection/builders/input_reader_builder_tf1_test.py to input_reader_builder_tf1_test.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/object_detection/builders/box_predictor_builder_test.py to box_predictor_builder_test.cpytho

byte-compiling build/bdist.linux-x86_64/egg/object_detection/protos/anchor_generator_pb2.py to anchor_generator_pb2.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/object_detection/protos/graph_rewriter_pb2.py to graph_rewriter_pb2.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/object_detection/protos/ssd_anchor_generator_pb2.py to ssd_anchor_generator_pb2.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/object_detection/protos/__init__.py to __init__.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/object_detection/protos/square_box_coder_pb2.py to square_box_coder_pb2.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/object_detection/protos/center_net_pb2.py to center_net_pb2.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/object_detection/protos/argmax_matcher_pb2.py to argmax_matcher_pb2.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/object_detection/protos/ssd_pb2.py to ssd_pb2.cpython-37.pyc
byte-compiling

byte-compiling build/bdist.linux-x86_64/egg/object_detection/box_coders/faster_rcnn_box_coder.py to faster_rcnn_box_coder.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/object_detection/box_coders/mean_stddev_box_coder_test.py to mean_stddev_box_coder_test.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/object_detection/exporter_lib_tf2_test.py to exporter_lib_tf2_test.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/object_detection/model_tpu_main.py to model_tpu_main.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/object_detection/export_tflite_ssd_graph_lib.py to export_tflite_ssd_graph_lib.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/object_detection/legacy/trainer_tf1_test.py to trainer_tf1_test.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/object_detection/legacy/evaluator.py to evaluator.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/object_detection/legacy/__init__.py to __init__.cpython-37.pyc
b

byte-compiling build/bdist.linux-x86_64/egg/object_detection/models/keras_models/__init__.py to __init__.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/object_detection/models/keras_models/convert_keras_models.py to convert_keras_models.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/object_detection/models/keras_models/inception_resnet_v2_tf2_test.py to inception_resnet_v2_tf2_test.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/object_detection/models/keras_models/hourglass_network_tf2_test.py to hourglass_network_tf2_test.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/object_detection/models/keras_models/mobilenet_v2.py to mobilenet_v2.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/object_detection/models/keras_models/mobilenet_v1_tf2_test.py to mobilenet_v1_tf2_test.cpython-37.pyc
byte-compiling build/bdist.linux-x86_64/egg/object_detection/models/keras_models/test_utils.py to test_utils.cpython-37.pyc
byte-compiling build/b

In [10]:
import sys,os,os.path
!pwd
os.environ['PYTHONPATH']='./:./slim'
!printenv

/home/subzero/Documents/0.TensorflowTrain/RaccoonDetector/models/research
LESSOPEN=| /usr/bin/lesspipe %s
CONDA_PROMPT_MODIFIER=(tf1gpu) 
USER=subzero
WORKON_HOME=/home/subzero/.virtualenvs
LC_TIME=vi_VN
MKL_THREADING_LAYER=INTEL
TEXTDOMAIN=im-config
XDG_SEAT=seat0
MPLBACKEND=module://ipykernel.pylab.backend_inline
SSH_AGENT_PID=3142
XDG_SESSION_TYPE=x11
LD_LIBRARY_PATH=/usr/local/cuda-10.1/lib64
SHLVL=1
QT4_IM_MODULE=xim
HOME=/home/subzero
CONDA_SHLVL=2
VIRTUALENVWRAPPER_PYTHON=/usr/bin/python3
DESKTOP_SESSION=ubuntu
__KMP_REGISTERED_LIB_5191=0x7fbc244c4708-cafecea3-libiomp5.so
GNOME_SHELL_SESSION_MODE=ubuntu
GTK_MODULES=gail:atk-bridge
PAGER=cat
MANAGERPID=3016
LC_MONETARY=vi_VN
DBUS_STARTER_BUS_TYPE=session
DBUS_SESSION_BUS_ADDRESS=unix:path=/run/user/1000/bus,guid=5c2c4eb4233e22acdbef32695ef29306
_CE_M=
COLORTERM=truecolor
__KMP_REGISTERED_LIB_5683=0x7fbc244c4708-cafe27dd-libiomp5.so
MANDATORY_PATH=/usr/share/gconf/ubuntu.mandatory.path
IM_CONFIG_PHASE=2
LOGNAME=subzero
GTK_IM_MODU

In [11]:
# testing the model builder
!python3 object_detection/builders/model_builder_test.py

# Generating Tf record

*   Generating two TFRecords files for the training and testing CSVs.
*   Tensorflow accepts the data as tfrecords which is a binary file that run fast with low memory usage. Instead of loading the full data into memory, Tenorflow breaks the data into batches using these TFRecords automatically





In [12]:
#adjusted from: https://github.com/datitran/raccoon_dataset
from object_detection.utils import dataset_util


#change this to the base directory where your data/ is 
data_base_url = '/gdrive/My Drive/raccoon/data/'

#location of images
image_dir = 'images/'

def class_text_to_int(row_label):
  if row_label == 'raccoon':
    return 1
  else:
    None

def split(df, group):
  data = namedtuple('data', ['filename', 'object'])
  gb = df.groupby(group)
  return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_example(group, path):
	with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
	  encoded_jpg = fid.read()
	encoded_jpg_io = io.BytesIO(encoded_jpg)
	image = Image.open(encoded_jpg_io)
	width, height = image.size
	filename = group.filename.encode('utf8')
	image_format = b'jpg'
	xmins = []
	xmaxs = []
	ymins = []
	ymaxs = []
	classes_text = []
	classes = []

	for index, row in group.object.iterrows():
		xmins.append(row['xmin'] / width)
		xmaxs.append(row['xmax'] / width)
		ymins.append(row['ymin'] / height)
		ymaxs.append(row['ymax'] / height)
		classes_text.append(row['class'].encode('utf8'))
		classes.append(class_text_to_int(row['class']))

	tf_example = tf.train.Example(features=tf.train.Features(feature={
		'image/height': dataset_util.int64_feature(height),
		'image/width': dataset_util.int64_feature(width),
		'image/filename': dataset_util.bytes_feature(filename),
		'image/source_id': dataset_util.bytes_feature(filename),
		'image/encoded': dataset_util.bytes_feature(encoded_jpg),
		'image/format': dataset_util.bytes_feature(image_format),
		'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
		'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
		'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
		'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
		'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
		'image/object/class/label': dataset_util.int64_list_feature(classes),
		}))
	return tf_example


In [13]:
%cd /home/subzero/Documents/0.TensorflowTrain/RaccoonDetector/data
#creates tfrecord for both csv's

for csv in ['train_labels', 'test_labels']:
  print(data_base_url)
  writer = tf.io.TFRecordWriter(csv + '.record')
  path = os.path.join(image_dir)
  examples = pd.read_csv(csv + '.csv')
  grouped = split(examples, 'filename')
  for group in grouped:
    tf_example = create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())
    
  writer.close()
  output_path = os.path.join(os.getcwd(), csv + '.record')
  print('Successfully created the TFRecords: {}'.format(csv + '.record'))

/home/subzero/Documents/0.TensorflowTrain/RaccoonDetector/data
/gdrive/My Drive/raccoon/data/
Successfully created the TFRecords: train_labels.record
/gdrive/My Drive/raccoon/data/
Successfully created the TFRecords: test_labels.record


# Selecting and Downloading a Pre-trained Model

In [38]:
!pwd

/home/subzero/Documents/0.TensorflowTrain/RaccoonDetector/models/research


In [39]:
# Some models to train on
MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
    },
}

# Select a model from `MODELS_CONFIG`.
# I chose ssd_mobilenet_v2 for this project, you could choose any
selected_model = 'ssd_mobilenet_v2'

In [40]:
#the distination folder where the model will be saved
#change this if you have a different working dir
DEST_DIR = '/home/subzero/Documents/0.TensorflowTrain/RaccoonDetector/models/research/pretrained_model'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

#selecting the model
MODEL_FILE = MODEL + '.tar.gz'

#creating the downlaod link for the model selected
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

#checks if the model has already been downloaded, download it otherwise
if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

#unzipping the model and extracting its content
tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

# creating an output file to save the model while training
os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

# Configuring the Training Pipeline



```
Edit config file 
*   model {} > ssd {}: change num_classes to the number of classes you have.
*   train_config {}: change fine_tune_checkpoint to the checkpoint file path.
*   train_input_reader {}: set the path to the train_labels.record and the label map pbtxt file.
*   eval_input_reader {}: set the path to the test_labels.record and the label map pbtxt file
```

In [ ]:
#path to the config file
!cat object_detection/samples/configs/ssd_mobilenet_v2_coco.config

# Tensorboard

To use Tensorboard on Colab, we need to use it through ngrok. Get it by running

In [14]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

--2020-06-24 07:29:34--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.2.226.158, 54.164.74.108, 35.175.20.97, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.2.226.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13,13M  2,73MB/s    in 6,1s    

2020-06-24 07:29:41 (2,15 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


specify where the log files are stored and we configure a link to view Tensorboard

In [15]:
%cd /home/subzero/Documents/0.TensorflowTrain/RaccoonDetector/models/research/training
#the logs that are created while training 
LOG_DIR = "training/"
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
#The link to tensorboard.
#works after the training starts.
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

/home/subzero/Documents/0.TensorflowTrain/RaccoonDetector/models/research/training
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/subzero/anaconda3/envs/tf1gpu/lib/python3.7/json/__init__.py", line 296, in load
    parse_constant=parse_constant, object_pairs_hook=object_pairs_hook, **kw)
  File "/home/subzero/anaconda3/envs/tf1gpu/lib/python3.7/json/__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "/home/subzero/anaconda3/envs/tf1gpu/lib/python3.7/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/home/subzero/anaconda3/envs/tf1gpu/lib/python3.7/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


# Training
Add to train.py:

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [19]:
%cd /home/subzero/Documents/0.TensorflowTrain/RaccoonDetector/models/research

/home/subzero/Documents/0.TensorflowTrain/RaccoonDetector/models/research
/home/subzero/Documents/0.TensorflowTrain/RaccoonDetector/models/research


In [16]:
physical_devices = tf.config.experimental.list_physical_devices('GPU') 
for physical_device in physical_devices: 
    tf.config.experimental.set_memory_growth(physical_device, True)

In [20]:
!python3 object_detection/legacy/train.py --logtostderr --train_dir=training/ \
    --pipeline_config_path=object_detection/samples/configs/ssd_mobilenet_v2_coco.config

2020-06-24 07:33:06.635889: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2020-06-24 07:33:06.833838: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2899885000 Hz
2020-06-24 07:33:06.834665: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5585af6ad430 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-06-24 07:33:06.835068: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-06-24 07:33:06.843765: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-06-24 07:33:06.850671: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-06-24

Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.
W0624 07:33:09.380922 139885348169536 deprecation.py:323] From /home/subzero/Documents/0.TensorflowTrain/RaccoonDetector/models/research/object_detection/core/preprocessor.py:198: sample_distorted_bounding_box (from tensorflow.python.ops.image_ops_impl) is deprecated and will be removed in a future version.
Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0624 07:33:09.395692 139885348169536 deprecation.py:323] From /home/subzero/Documents/0.TensorflowTrain/RaccoonDetector/models/research/object_detection/core/box_list_ops.py:231: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating

W0624 07:33:18.577730 139885348169536 variables_helper.py:156] Variable [FeatureExtractor/MobilenetV2/expanded_conv_2/project/BatchNorm/beta/RMSProp_1] is not available in checkpoint
W0624 07:33:18.577761 139885348169536 variables_helper.py:156] Variable [FeatureExtractor/MobilenetV2/expanded_conv_2/project/BatchNorm/gamma/ExponentialMovingAverage] is not available in checkpoint
W0624 07:33:18.577789 139885348169536 variables_helper.py:156] Variable [FeatureExtractor/MobilenetV2/expanded_conv_2/project/BatchNorm/gamma/RMSProp] is not available in checkpoint
W0624 07:33:18.577817 139885348169536 variables_helper.py:156] Variable [FeatureExtractor/MobilenetV2/expanded_conv_2/project/BatchNorm/gamma/RMSProp_1] is not available in checkpoint
W0624 07:33:18.577853 139885348169536 variables_helper.py:156] Variable [FeatureExtractor/MobilenetV2/expanded_conv_2/project/weights/ExponentialMovingAverage] is not available in checkpoint
W0624 07:33:18.577881 139885348169536 variables_helper.py:156

Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
W0624 07:33:19.147904 139885348169536 deprecation.py:323] From /home/subzero/anaconda3/envs/tf1gpu/lib/python3.7/site-packages/tf_slim/learning.py:734: Supervisor.__init__ (from tensorflow.python.training.supervisor) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
2020-06-24 07:33:19.924211: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-06-24 07:33:19.924465: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: GeForce RTX 2060 SUPER major: 7 minor: 5 memoryClockRate(GHz): 1.665
pciBusID: 0000:01:00.0
2020-06-24 07:33:19.924513: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudar

INFO:tensorflow:global step 20: loss = 7.4943 (0.070 sec/step)
I0624 07:33:56.333583 139885348169536 learning.py:512] global step 20: loss = 7.4943 (0.070 sec/step)
INFO:tensorflow:global step 21: loss = 8.5703 (0.078 sec/step)
I0624 07:33:56.412409 139885348169536 learning.py:512] global step 21: loss = 8.5703 (0.078 sec/step)
INFO:tensorflow:global step 22: loss = 7.1395 (0.068 sec/step)
I0624 07:33:56.481608 139885348169536 learning.py:512] global step 22: loss = 7.1395 (0.068 sec/step)
INFO:tensorflow:global step 23: loss = 6.4413 (0.069 sec/step)
I0624 07:33:56.551842 139885348169536 learning.py:512] global step 23: loss = 6.4413 (0.069 sec/step)
INFO:tensorflow:global step 24: loss = 7.4912 (0.071 sec/step)
I0624 07:33:56.623688 139885348169536 learning.py:512] global step 24: loss = 7.4912 (0.071 sec/step)
INFO:tensorflow:global step 25: loss = 7.1455 (0.078 sec/step)
I0624 07:33:56.702764 139885348169536 learning.py:512] global step 25: loss = 7.1455 (0.078 sec/step)
INFO:tenso

...

INFO:tensorflow:global step 7479: loss = 1.5344 (0.075 sec/step)
I0624 07:43:14.416188 139885348169536 learning.py:512] global step 7479: loss = 1.5344 (0.075 sec/step)
INFO:tensorflow:global step 7480: loss = 1.5745 (0.079 sec/step)
I0624 07:43:14.496689 139885348169536 learning.py:512] global step 7480: loss = 1.5745 (0.079 sec/step)
INFO:tensorflow:global step 7481: loss = 2.8100 (0.069 sec/step)
I0624 07:43:14.566595 139885348169536 learning.py:512] global step 7481: loss = 2.8100 (0.069 sec/step)
INFO:tensorflow:global step 7482: loss = 1.6837 (0.081 sec/step)
I0624 07:43:14.648127 139885348169536 learning.py:512] global step 7482: loss = 1.6837 (0.081 sec/step)
INFO:tensorflow:global step 7483: loss = 1.3313 (0.076 sec/step)
I0624 07:43:14.725399 139885348169536 learning.py:512] global step 7483: loss = 1.3313 (0.076 sec/step)
INFO:tensorflow:global step 7484: loss = 2.5617 (0.074 sec/step)
I0624 07:43:14.800590 139885348169536 learning.py:512] global step 7484: loss = 2.5617 (0.

INFO:tensorflow:global step 7528: loss = 0.8457 (0.087 sec/step)
I0624 07:43:18.242684 139885348169536 learning.py:512] global step 7528: loss = 0.8457 (0.087 sec/step)
INFO:tensorflow:global step 7529: loss = 1.8560 (0.080 sec/step)
I0624 07:43:18.323307 139885348169536 learning.py:512] global step 7529: loss = 1.8560 (0.080 sec/step)
INFO:tensorflow:global step 7530: loss = 2.5627 (0.083 sec/step)
I0624 07:43:18.407430 139885348169536 learning.py:512] global step 7530: loss = 2.5627 (0.083 sec/step)
INFO:tensorflow:global step 7531: loss = 2.0483 (0.090 sec/step)
I0624 07:43:18.498842 139885348169536 learning.py:512] global step 7531: loss = 2.0483 (0.090 sec/step)
INFO:tensorflow:global step 7532: loss = 1.9293 (0.081 sec/step)
I0624 07:43:18.580572 139885348169536 learning.py:512] global step 7532: loss = 1.9293 (0.081 sec/step)
INFO:tensorflow:global step 7533: loss = 3.0445 (0.077 sec/step)
I0624 07:43:18.658809 139885348169536 learning.py:512] global step 7533: loss = 3.0445 (0.

INFO:tensorflow:global step 7577: loss = 3.2583 (0.084 sec/step)
I0624 07:43:22.089038 139885348169536 learning.py:512] global step 7577: loss = 3.2583 (0.084 sec/step)
INFO:tensorflow:global step 7578: loss = 1.9031 (0.074 sec/step)
I0624 07:43:22.164199 139885348169536 learning.py:512] global step 7578: loss = 1.9031 (0.074 sec/step)
INFO:tensorflow:global step 7579: loss = 2.1259 (0.072 sec/step)
I0624 07:43:22.237753 139885348169536 learning.py:512] global step 7579: loss = 2.1259 (0.072 sec/step)
INFO:tensorflow:global step 7580: loss = 2.2741 (0.067 sec/step)
I0624 07:43:22.305608 139885348169536 learning.py:512] global step 7580: loss = 2.2741 (0.067 sec/step)
INFO:tensorflow:global step 7581: loss = 1.8283 (0.081 sec/step)
I0624 07:43:22.388170 139885348169536 learning.py:512] global step 7581: loss = 1.8283 (0.081 sec/step)
INFO:tensorflow:global step 7582: loss = 0.9850 (0.070 sec/step)
I0624 07:43:22.459837 139885348169536 learning.py:512] global step 7582: loss = 0.9850 (0.

INFO:tensorflow:global step 7626: loss = 1.2679 (0.067 sec/step)
I0624 07:43:25.728193 139885348169536 learning.py:512] global step 7626: loss = 1.2679 (0.067 sec/step)
INFO:tensorflow:global step 7627: loss = 2.1154 (0.080 sec/step)
I0624 07:43:25.809482 139885348169536 learning.py:512] global step 7627: loss = 2.1154 (0.080 sec/step)
INFO:tensorflow:global step 7628: loss = 1.7055 (0.072 sec/step)
I0624 07:43:25.882469 139885348169536 learning.py:512] global step 7628: loss = 1.7055 (0.072 sec/step)
INFO:tensorflow:global step 7629: loss = 1.1666 (0.072 sec/step)
I0624 07:43:25.956001 139885348169536 learning.py:512] global step 7629: loss = 1.1666 (0.072 sec/step)
INFO:tensorflow:global step 7630: loss = 1.7875 (0.070 sec/step)
I0624 07:43:26.026676 139885348169536 learning.py:512] global step 7630: loss = 1.7875 (0.070 sec/step)
INFO:tensorflow:global step 7631: loss = 1.5039 (0.076 sec/step)
I0624 07:43:26.103507 139885348169536 learning.py:512] global step 7631: loss = 1.5039 (0.

INFO:tensorflow:global step 7674: loss = 2.7293 (0.073 sec/step)
I0624 07:43:29.567027 139885348169536 learning.py:512] global step 7674: loss = 2.7293 (0.073 sec/step)
INFO:tensorflow:global step 7675: loss = 1.6313 (0.080 sec/step)
I0624 07:43:29.649124 139885348169536 learning.py:512] global step 7675: loss = 1.6313 (0.080 sec/step)
INFO:tensorflow:global step 7676: loss = 3.5031 (0.094 sec/step)
I0624 07:43:29.745402 139885348169536 learning.py:512] global step 7676: loss = 3.5031 (0.094 sec/step)
INFO:tensorflow:global step 7677: loss = 4.6003 (0.180 sec/step)
I0624 07:43:29.930578 139885348169536 learning.py:512] global step 7677: loss = 4.6003 (0.180 sec/step)
INFO:tensorflow:global step 7678: loss = 1.7236 (0.111 sec/step)
I0624 07:43:30.053435 139885348169536 learning.py:512] global step 7678: loss = 1.7236 (0.111 sec/step)
INFO:tensorflow:global step 7679: loss = 2.7082 (0.094 sec/step)
I0624 07:43:30.164103 139885348169536 learning.py:512] global step 7679: loss = 2.7082 (0.

In [9]:
!pwd

/gdrive/My Drive/Data/raccoon/models/research
